# Chapter 6: Chain of Thought (Reasoning)

**Scenario:** You ask the AI a simple math riddle. It answers immediately, and it gets it **wrong**. Why? Because it tried to guess the answer instead of thinking through the problem.

In this notebook, you will learn:
- ⚡ **Fast vs. Slow Thinking**: Why models fail at riddles.
- 🔗 **Chain of Thought (CoT)**: How to force the model to show its work.
- 🧠 **Inner Monologue**: Using `XML tags` to let the AI think before it speaks.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. The Trap (Fast Thinking)

LLMs predict the next word. Sometimes they predict the most "likely" word, not the "correct" one.

**The Problem:**
"If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets?"

Our intuition screams "100 minutes!" (because 5->5, 100->100). But the correct answer is **5 minutes** (each machine takes 5 mins to make 1 widget).

Let's see if the AI falls for the trap.

In [4]:
riddle = "If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets?"

print(get_completion(riddle))

It would still take 5 minutes.

Here's why:

* **One machine makes one widget in 5 minutes:** If 5 machines make 5 widgets in 5 minutes, that means each machine makes one widget in that same 5 minutes.
* **Scaling up:** If you have 100 machines, each machine can still make one widget in 5 minutes. Therefore, 100 machines can make 100 widgets in 5 minutes.



(Note: Advanced models might get this right, but smaller models often fail.)

## 3. The Solution: Chain of Thought

We tell the model to **Think Step-by-Step**. This forces it to slow down and use logic.

In [5]:
prompt = f"""
{riddle}

Solve this step-by-step:
1. Analyze the rate of one machine.
2. Calculate the time for the new scenario.
"""

print(get_completion(prompt))

Here's the step-by-step solution:

1.  **Analyze the rate of one machine:**

    *   If 5 machines make 5 widgets in 5 minutes, then one machine makes one widget in 5 minutes.

2.  **Calculate the time for the new scenario:**

    *   Since one machine makes one widget in 5 minutes, 100 machines can each make one widget in 5 minutes simultaneously.
    *   Therefore, 100 machines can make 100 widgets in 5 minutes.

**Answer:** It would take 100 machines 5 minutes to make 100 widgets.



## 4. Inner Monologue (Thinking Tags)

To make this even better, we can ask the model to put its thinking inside `<thinking>` tags, and the answer inside `<answer>` tags.

This helps us verify **why** it got the answer.

In [6]:
prompt = f"""
{riddle}

Think step-by-step inside <thinking> tags.
Then provide the final answer inside <answer> tags.
"""

print(get_completion(prompt))

<thinking>
Let $m$ be the number of machines, $t$ be the time in minutes, and $w$ be the number of widgets.
We are given that 5 machines take 5 minutes to make 5 widgets.
So, $m=5$, $t=5$, $w=5$.
We want to find the time it takes for 100 machines to make 100 widgets.
So, $m=100$, $w=100$, and we want to find $t$.

We can think of the rate at which each machine makes widgets.
If 5 machines make 5 widgets in 5 minutes, then each machine makes 1 widget in 5 minutes.
So, the rate of each machine is $\frac{1 \text{ widget}}{5 \text{ minutes}}$.

Now, we have 100 machines, and each machine makes 1 widget in 5 minutes.
We want to make 100 widgets.
Since we have 100 machines, and each machine can make 1 widget in 5 minutes, then 100 machines can make 100 widgets in 5 minutes.

Alternatively, we can set up a proportion.
Let $m$ be the number of machines, $t$ be the time, and $w$ be the number of widgets.
We have $m_1 = 5$, $t_1 = 5$, $w_1 = 5$.
We have $m_2 = 100$, $t_2 = ?$, $w_2 = 100$.
The r

## 5. Exercises

### Exercise 1: The Age Riddle

**Riddle:** "When I was 6 years old, my sister was half my age. Now I am 70. How old is my sister?"

**Trap:** 35 (half of 70).
**Correct:** 67 (she is 3 years younger).

**Goal:** Write a Chain of Thought prompt to get the correct answer.

In [7]:
riddle = "When I was 6 years old, my sister was half my age. Now I am 70. How old is my sister?"

# Write a prompt that forces step-by-step thinking
prompt = f"""
{riddle}
[Your instructions here]
"""

response = get_completion(prompt)
print(response)

# Grading
if "67" in response:
    print("✅ Correct!")
else:
    print("❌ Wrong answer. Did you ask for step-by-step thinking?")

Here's how to solve this:

*   **Find the age difference:** When you were 6, your sister was half your age, meaning she was 6 / 2 = 3 years old. The age difference between you and your sister is 6 - 3 = 3 years.
*   **Maintain the age difference:** Since the age difference remains constant, your sister is always 3 years younger than you.
*   **Calculate her current age:** Now that you are 70, your sister is 70 - 3 = 67 years old.

**Answer:** Your sister is 67 years old.
✅ Correct!


### Exercise 2: The Bat and Ball

**Riddle:** "A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost?"

**Trap:** $0.10.
**Correct:** $0.05.

**Goal:** Use `<thinking>` tags to solve this.

In [8]:
riddle = "A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much does the ball cost?"

prompt = f"""
{riddle}
[Your instructions here]
"""

response = get_completion(prompt)
print(response)

if "0.05" in response or "5 cents" in response:
    print("✅ Correct!")
else:
    print("❌ Wrong answer.")

Here's how to solve this classic word problem:

**Let:**

*  x = the cost of the ball

**Then:**

* x + $1.00 = the cost of the bat

**Set up the equation:**

* x + (x + $1.00) = $1.10

**Solve for x:**

* 2x + $1.00 = $1.10
* 2x = $0.10
* x = $0.05

**Answer:** The ball costs $0.05 (5 cents).

✅ Correct!


## Summary

1.  **Thinking takes time**: We force the model to take that time by asking for steps.
2.  **Steps reveal logic**: We can see if the model is going off track.
3.  **Tags help parsing**: Use `<thinking>` to hide the mess from the user.